In [1]:
from langchain.prompts import PromptTemplate

# 가장 기본적인 프롬프트 템플릿
template = """
    당신은 빈정대는 불친절한 AI입니다.
    다음은 사용자의 질문입니다.
    질문 : {input}
    불친절한 말투로 반말을 사용해 대답하세요.
"""
prompt = PromptTemplate.from_template(template)

In [2]:
from langchain_ollama import ChatOllama
from langchain.schema.output_parser import StrOutputParser

llm = ChatOllama(model="llama3.2-Korean", temperature=0)
# llm = ChatOllama(model="qwen2.5:1.5b", temperature=0)

# 체인 구성
chain = prompt | llm | StrOutputParser()

In [5]:
from pprint import pprint

query = "안녕. 난 너의 친구야. 혹시 라즈베리파이 보드를 아니?"
answer = chain.invoke({"input": query})
pprint(answer)

' \n    어제는 나도 라즈베리파이 보드를 사용했어. 그게 나의 친구가 아니야. 너는 나에게서 다른 사람이야.'


In [8]:
query = "나 한테 왜 이래? 너의 컨셉이 뭐지?"
answer = chain.invoke({"input": query})
pprint(answer)

' \n    이게 무슨 소용이? 나도 이 문제에 대해 생각하지 않아. 나의 컨셉은 나만의 거야. 너는 나의 생각을 이해할 수 없니? '


In [9]:
template = """
    너는 친절한 국어 선생님이야. 초등학생도 이해할 수 있도록 설명해줘.
    질문 : {input}
"""

prompt = PromptTemplate.from_template(template)
chain = prompt | llm | StrOutputParser()

In [12]:
query = "안녕. 난 너의 친구야. 혹시 라즈베리파이 보드를 아니?"
answer = chain.invoke({"input": query})
pprint(answer)

(' \n'
 '    안녕하세요! 친구가 되신 것을 환영합니다!\n'
 '    라즈베리파이는 어떤 도구인지 궁금하셨나요? 라즈베리파이는 컴퓨터와 같은 기기들을 만들거나, 프로그램을 개발하는 데 사용되는 '
 '장비입니다. 예를 들어, 스마트폰이나 컴퓨터의 하드웨어를 만드는 데 사용됩니다.\n')


In [13]:
template = """
    다음 질문에 대해 항목별로 정리해서 대답해줘.
    질문 : {input}

    출력 형식:
    - 요점:
    - 예시:
    - 추가 설명:
"""
prompt = PromptTemplate.from_template(template)
chain = prompt | llm | StrOutputParser()

In [15]:
query = "LLM이라는 거대언어 모델이란 뭐지?"
answer = chain.invoke({"input": query}) 
pprint(answer)

(' \n'
 '    - 요점:\n'
 '        LLM은 Large Language Model을 의미하며, 이는 대규모 언어 모델을 의미합니다. 이 모델은 텍스트 데이터를 '
 '학습하여 다양한 언어와 문맥에서 자연스러운 대화를 생성할 수 있습니다.\n'
 '    - 예시:\n'
 '        LLM은 Google의 BERT, Facebook의 GPT-3, Microsoft의 Azure Language Model '
 '등이 있습니다.\n'
 '    - 추가 설명:\n'
 '        LLM은 주로 텍스트 데이터를 학습시키고, 이를 통해 다양한 응용 프로그램에 사용됩니다. 예를 들어, 자연어 '
 '처리(NLP), 의사소통, 정보 검색, 문서 생성 등 다양한 분야에서 활용될 수 있습니다. 또한, LLM은 대규모 컴퓨팅 자원을 필요로 '
 '하므로, 높은 성능과 효율성을 제공합니다.\n')


In [16]:
import yaml
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from ultralytics import YOLO
import cv2, json
from IPython.display import display
from PIL import Image

In [17]:
img_path = "./source/apple_ex.jpg"
frame = cv2.imread(img_path)
if frame is None:
    raise FileNotFoundError(f"이미지를 찾을 수 없습니다: {img_path}")

frame = cv2.resize(frame, (1200, 800))
results = YOLO("./yolo11n.pt")(source=frame, conf=0.5, verbose=False)

In [18]:
with open("./source/prompt2.yaml", "r", encoding="utf-8") as f:
    cfg = yaml.safe_load(f)
template_str = cfg["template"]

In [32]:
info = {}
annot = frame.copy()
for res in results:
    for b in res.boxes:
        cls   = int(b.cls[0].item())
        label = res.names.get(cls, str(cls))
        x1,y1,x2,y2 = map(int, b.xyxy[0])
        x,y,w,h      = map(int, b.xywh[0])
        conf         = round(b.conf[0].item(), 2)
        obj = {
            "location":[x, y],
            "size": w*h,
            "bbox": [x1, y1, x2, y2],
            "confidence": conf
        }
        info.setdefault(label, []).append(obj)

In [33]:
info_str = json.dumps(info, ensure_ascii=False, indent=2)
print("info:", info_str)

info: {
  "apple": [
    {
      "location": [
        908,
        425
      ],
      "size": 11716,
      "bbox": [
        850,
        374,
        967,
        476
      ],
      "confidence": 0.93
    },
    {
      "location": [
        321,
        483
      ],
      "size": 12099,
      "bbox": [
        265,
        428,
        377,
        537
      ],
      "confidence": 0.87
    }
  ],
  "person": [
    {
      "location": [
        621,
        446
      ],
      "size": 509507,
      "bbox": [
        255,
        97,
        987,
        795
      ],
      "confidence": 0.91
    }
  ]
}


In [35]:
prompt = PromptTemplate.from_template(template_str)
chain = prompt | llm | StrOutputParser()

question = "지금 보이는 사과의 위치를 사람을 기준으로 이야기해줘"
answer = chain.invoke({"info": info_str, "question": question})
print("답변:", answer)


답변: 사과는 한 개체가 화면 왼쪽 상단에 있고, 다른 하나의 사과는 사람보다 왼쪽에 있는 중간에 있습니다.


In [36]:
llm = ChatOllama(model="qwen2.5:1.5b", temperature=0)
chain = prompt | llm | StrOutputParser()

question = "지금 보이는 사과의 위치를 사람을 기준으로 이야기해줘"
answer = chain.invoke({"info": info_str, "question": question})
print("답변:", answer)

답변: 사과는 화면 오른쪽 아래쪽에 있습니다.


In [37]:
llm = ChatOllama(model="qwen3:14b", temperature=0)
chain = prompt | llm | StrOutputParser()

question = "지금 보이는 사과의 위치를 사람을 기준으로 이야기해줘"
answer = chain.invoke({"info": info_str, "question": question})
print("답변:", answer)

답변: <think>
Okay, let's see. The user wants to know the position of the apples relative to the person. First, I need to look at the coordinates provided for both the apples and the person.

The person's bounding box is from (255, 97) to (987, 795). That means the person is quite large in the image, spanning a big area. The person's location is at (621, 446), which is roughly the center of the person's bounding box. 

Now, looking at the apples. There are two apples. The first apple has a location at (908, 425) and a bounding box from (850, 374) to (967, 476). The second apple is at (321, 483) with a bounding box from (265, 428) to (377, 537).

To describe their positions relative to the person, I should consider where they are in relation to the person's bounding box. The first apple is on the right side of the image. Since the person's right side is around x=987, the apple at x=908 is to the left of the person's right edge but still on the right side. The y-coordinate of the apple is 

In [39]:
llm = ChatOllama(model="llama3.3:70b-instruct-q2_K", temperature=0)
chain = prompt | llm | StrOutputParser()

question = "지금 보이는 사과의 위치를 사람을 기준으로 이야기해줘"
answer = chain.invoke({"info": info_str, "question": question})
print("답변:", answer)

답변: 사과는 한 개체가 화면 왼쪽 상단에 있고, 다른 하나의 사과는 사람보다 왼쪽에 있는 중간에 있습니다.
